In [1]:
from stockfish import Stockfish
import chess

stockfish_path = "/Users/benitorusconi/Documents/CDS/05_HS23/Reinforcement Learning (cds-117)/engine/stockfish"
stockfish = Stockfish(path=stockfish_path)

def print_board(board):
    print(board)

def play_game():
    board = chess.Board()

    print("Chess game against Stockfish\n")

    while not board.is_game_over():
        print_board(board)

        # Player's move
        player_move = input("Your move (in algebraic notation): ")
        if chess.Move.from_uci(player_move) in board.legal_moves:
            board.push_uci(player_move)
        else:
            print("Invalid move. Try again.")
            continue

        if board.is_game_over():
            break

        # Stockfish's move
        stockfish.set_fen_position(board.fen())
        stockfish_move = stockfish.get_best_move()
        print("Stockfish's move:", stockfish_move)
        board.push_uci(stockfish_move)

    print("\nGame Over")
    print("Result:", board.result())

# Play the game
#play_game()


In [26]:
from stockfish import Stockfish
import chess
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.compat.v1.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from IPython.display import display, HTML
import chess.svg

stockfish_path = "/Users/benitorusconi/Documents/CDS/05_HS23/Reinforcement Learning (cds-117)/engine/stockfish"
stockfish = Stockfish(path=stockfish_path)

learning_rate = 0.001
discount_factor = 0.9
exploration_prob = 0.2

state_space_size = 64
action_space_size = 4096
q_table = np.zeros((state_space_size, action_space_size))

model = Sequential([
    Dense(64, input_shape=(state_space_size,), activation='relu'),
    Dense(128, activation='relu'),
    Dense(action_space_size, activation='linear')
])

model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')

def state_to_index(board):
    return hash(board.fen()) % state_space_size

def choose_action(board):
    if np.random.rand() < exploration_prob:
        return np.random.choice(list(board.legal_moves))
    else:
        state_index = state_to_index(board)
        legal_moves_list = list(board.legal_moves)
        if not legal_moves_list:
            return chess.Move.null()
        best_move_index = np.argmax(q_table[state_index])
        best_move_uci = legal_moves_list[min(best_move_index, len(legal_moves_list)-1)].uci()
        return chess.Move.from_uci(best_move_uci)

def update_q_table(state, action, reward, next_state):
    state_index = state_to_index(state)
    next_state_index = state_to_index(next_state)
    action_index = list(state.legal_moves).index(action)
    best_next_action = np.argmax(q_table[next_state_index])
    q_table[state_index, action_index] += learning_rate * (
        reward + discount_factor * q_table[next_state_index, best_next_action] - q_table[state_index, action_index]
    )

def display_chess_board(board):
    return display(HTML(chess.svg.board(board=board, size=400)))

def play_game():
    board = chess.Board()
    game_states = []

    while not board.is_game_over():
        state = board.copy()
        game_states.append(state.copy())

        rl_move = choose_action(board)
        if rl_move in board.legal_moves:
            board.push(rl_move)
        else:
            print("Invalid move. Try again.")
            continue

        reward = 0

        if board.is_game_over():
            break

        stockfish.set_fen_position(board.fen())
        stockfish_move_uci = stockfish.get_best_move()
        stockfish_move = chess.Move.from_uci(stockfish_move_uci)
        next_state = board.copy()
        board.push(stockfish_move)

        if next_state.is_check():
            reward = 0.5

        if board.result() == "1-0":
            reward = 1e6
        elif board.result() == "0-1":
            reward = -1e6
        elif board.result() == "1/2-1/2":
            reward = 0.1
        elif board.is_capture(rl_move):
            reward = 1
        elif board.is_capture(stockfish_move):
            reward = -1

        update_q_table(state, rl_move, reward, next_state)

    game_states.append(board.copy())
    return game_states, board.result()

log_dir = "logs/"
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

num_games = 10
for episode in range(num_games):
    print("Game:", episode)
    total_reward = 0
    steps = 0
    game_states, result = play_game()

    with tf.summary.create_file_writer(log_dir).as_default():
        tf.summary.scalar('Total Reward', total_reward, step=episode)
        tf.summary.scalar('Steps', steps, step=episode)
        tf.summary.flush()

        for layer in model.layers:
            for weight in layer.weights:
                tf.summary.histogram(weight.name, weight, step=episode)

    if episode == num_games - 1:
        for state in game_states:
            display_chess_board(state)


Game: 0
Game: 1
Game: 2
Game: 3
Game: 4
Game: 5
Game: 6
Game: 7
